In [1]:
import openai
import os
import requests
import json
from dotenv import load_dotenv
load_dotenv()



True

In [2]:
from openai import OpenAI
client=OpenAI(
api_key=os.environ.get("OPENAI_API_KEY")
)


In [3]:
completion=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages= [
        {"role":"system", "content": "you are helpful assistant, which give details about real time of different flights."},
        {"role":"user", "content":"tell me the flight details from Jaipur to Delhi for today "}
       ],
     max_tokens=50, 
     temperature=0.8
)

In [8]:
completion

ChatCompletion(id='chatcmpl-9Mag6YC29JyqBjPuQnSaW1AYXLTCY', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"start_location":"Jaipur","end_location":"Delhi"}', name='get_flight_details'), tool_calls=None))], created=1715171234, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=107, total_tokens=130))

In [4]:
output=completion.choices[0].message.content
print(output)

I'm sorry, but I don't have real-time access to flight information. However, I recommend checking with the airline or using a flight-tracking website or app for the most up-to-date information on flights from Jaipur to Delhi today.


In [5]:
function_description =[
             
                    {
                "name": "get_flight_details",
                "description": "Get a real time flight details",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "start_location": {
                            "type": "string",
                            "description": "The departure airport, e.g. Delhi, DEL",
                        },
                        "end_location": {
                            "type": "string",
                            "description": "The destination airport, e.g. Jaipur, JAI",
                        },
                    },
                    "required": ["start_location", "end_location"],
                },
            },
        
     ] 

In [11]:
user_prompt="tell me the flight details from Udaipur to Delhi for today "
completion=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages= [
        {"role":"system", "content": "you are helpful assistant, which give details about real time of different flights."},
        {"role":"user", "content":user_prompt}
       ],
     function_call="auto",
     functions=function_description, 
     max_tokens=50
)

In [7]:
completion

ChatCompletion(id='chatcmpl-9Mag6YC29JyqBjPuQnSaW1AYXLTCY', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"start_location":"Jaipur","end_location":"Delhi"}', name='get_flight_details'), tool_calls=None))], created=1715171234, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=107, total_tokens=130))

In [12]:
output=completion.choices[0].message
print(output)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"start_location":"Udaipur","end_location":"Delhi"}', name='get_flight_details'), tool_calls=None)


In [13]:
def get_flight_details(start_location, end_location):
    import requests

    url = "https://sky-scanner3.p.rapidapi.com/get-config"

    headers = {
	"X-RapidAPI-Key": "66e16bab9emsh9ac9ebfdc2f54dbp13176bjsn2693206354cd",
	"X-RapidAPI-Host": "sky-scanner3.p.rapidapi.com"
     }

    response = requests.get(url, headers=headers)

    print(response.json())
    return response.json()

In [14]:
start=json.loads(output.function_call.arguments).get("start_location")
end=json.loads(output.function_call.arguments).get("end_location")
params=json.loads(output.function_call.arguments)


In [15]:
type(params)

dict

In [16]:
print(start)
print(end)
print(params)

Udaipur
Delhi
{'start_location': 'Udaipur', 'end_location': 'Delhi'}


In [18]:
chosen_function = eval(output.function_call.name)
flight = chosen_function(**params)
print(chosen_function)
print(flight)

{'data': [{'country': 'Andorra', 'market': 'AD', 'locale': 'es-ES', 'currencyTitle': 'Euro', 'currency': 'EUR', 'currencySymbol': '€', 'site': 'www.skyscanner.net'}, {'country': 'United Arab Emirates', 'market': 'AE', 'locale': 'en-US', 'currencyTitle': 'United Arab Emirates Dirham', 'currency': 'AED', 'currencySymbol': 'AED', 'site': 'www.skyscanner.ae'}, {'country': 'Afghanistan', 'market': 'AF', 'locale': 'en-GB', 'currencyTitle': 'Afghan Afghani', 'currency': 'AFN', 'currencySymbol': 'AFN', 'site': 'www.skyscanner.net'}, {'country': 'Antigua and Barbuda', 'market': 'AG', 'locale': 'en-GB', 'currencyTitle': 'East Caribbean Dollar', 'currency': 'XCD', 'currencySymbol': '$', 'site': 'www.skyscanner.net'}, {'country': 'Anguilla', 'market': 'AI', 'locale': 'en-GB', 'currencyTitle': 'East Caribbean Dollar', 'currency': 'XCD', 'currencySymbol': '$', 'site': 'www.skyscanner.net'}, {'country': 'Albania', 'market': 'AL', 'locale': 'en-GB', 'currencyTitle': 'Albanian Lek', 'currency': 'ALL', 

In [11]:
new_name=output.function_call.name
print(new_name)

get_flight_details


In [19]:
second_completion= client.chat.completions.create(
       model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": flight},
    ],
    functions=function_description,
 )
response = second_completion.choices[0].message.content
print(response)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[1].content[0]': expected an object, but got a string instead.", 'type': 'invalid_request_error', 'param': 'messages[1].content[0]', 'code': 'invalid_type'}}